## 参考
https://www.kaggle.com/tanulsingh077/how-to-become-leaf-doctor-with-deep-learning

# このコンペティションについて 

メラノーマに続いて、今年もまた古典的なコンピュータビジョンの分類問題が出題されました。CV を始めたばかりの人にとっては、このライブコンペに手を出してみて、最初の一歩を踏み出すことができる絶好の機会です。このコンテストでは、分類の精度が問われますが、それがどのくらいの頻度で起こるのでしょうか？

通常、実際に必要なのは、葉の医者になることであり、農家が感染性の葉を識別し、手頃なレートでそれらを治すのを助けることです 😛 。
 
# このノートブックについて

* いつものように、これは初心者向けのノートで、キャッサバの葉の病気に特化した葉の医者に効率的になれる方法をお伝えします 😛 そして、主な方法論はディープラーニングです。

* 私はあなたが知っておく必要があるすべてのものをカバーします , 専門知識から方法論まで , 私は問題を解決するために提案するさまざまなアイデアのベースラインの例と一緒に

* 多くの混乱がなければ、あなたはこのノートブックに従うと、これはあなたの最初のCV competitionを行うことができます。

* 機械学習とkaggleが全く初めての方は、私が書いたこの[guide](https://www.kaggle.com/tanulsingh077/tackling-any-kaggle-competition-the-noob-s-way) を見てみてください。 

# Step 1 : 患者の分析

* 彼のクライアントが話すことができないという事実を考慮して何かの前に葉の医者がすべきであることを最初のステップは何でしょうか？答えは当然簡単で、患者を見て何が間違っているかを分析します。

* しかし、医師はそれを見るだけで何かが間違っているかどうかをどのように理解しているのでしょうか？医師としてこのためには、彼は正常な患者/葉がどのように見えるかを知っている必要がありますし、感染したものから健康な患者を分離するために、通常の動作からの逸脱（パターン、色、質感など）を観察する必要があります。今、さらに病気の特定のクラスに感染したものを分類するために、医師はまた、患者/葉の状態が異なる疾患のように見える方法を知っておく必要があります。

これらのポイントを念頭に置いて、基本的な馴染みのあるものから始めてみましょう。

In [ ]:
import sys
#ライブラリをインポートする際にパスを追加
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
# Preliminaries
import os
from pathlib import Path
import glob
#プログレスバーを表示する
from tqdm import tqdm
tqdm.pandas()
import json
import pandas as pd
import numpy as np

## Image hash
import imagehash

# Visuals and CV2
import seaborn as sn
import matplotlib.pyplot as plt
import cv2
from PIL import Image


# albumentations for augs
# 画像の加工
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# クラスタリング、次元削減
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Keras and TensorFlow
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array 
from keras.applications.resnet50 import preprocess_input 

# models 
from keras.applications.resnet50 import ResNet50
from keras.models import Model

#torch
import torch
import timm
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader

# Utils

ユーティリティー機能の項

In [ ]:
def plot_images(class_id, label, images_number,verbose=0):
    '''
    Courtesy of https://www.kaggle.com/isaienkov/cassava-leaf-disease-classification-data-analysis
    '''
    '''
    ラベルがclass_idの画像をimages_number枚ランダムで取得し表示
    '''
    plot_list = train[train["label"] == class_id].sample(images_number)['image_id'].tolist()
    
    # 画像のリストを表示
    if verbose:
        print(plot_list)
        
    labels = [label for i in range(len(plot_list))]
    size = np.sqrt(images_number)
    #画像をsubplotを使って複数表示するために、sizeという変数をうまく設定している
    if int(size)*int(size) < images_number:
        size = int(size) + 1
        
    plt.figure(figsize=(20, 20))
    
    for ind, (image_id, label) in enumerate(zip(plot_list, labels)):
        plt.subplot(size, size, ind + 1)
        image = cv2.imread(str(BASE_DIR/'train_images'/image_id))
        #OpenCVでは画像をBGRの順で読み込むので変換する必要がある。
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(label, fontsize=12)
        plt.axis("off")
    
    plt.show()

In [ ]:
BASE_DIR = Path('../input/cassava-leaf-disease-classification')

## Reading DataFrame having Labels
train = pd.read_csv(BASE_DIR/'train.csv')

## Label Mappings
with open(BASE_DIR/'label_num_to_disease_map.json') as f:
    mapping = json.loads(f.read())
    mapping = {int(k): v for k,v in mapping.items()}

print(mapping)

<b>このdoctoeのコースでは、4つの病気について学ぶ必要がありますが、その前に、これらの病気の名前をデータセットのラベルにマッピングすることができます。 </b>

In [ ]:
train['label_names'] = train['label'].map(mapping)
train.head()

## Step 1.1 健康なもの(Healthy)を学ぶ

今、私達は健康なイメージを見始め、健康なキャッサバの葉の特徴の私達の理解を形成することができる1つの場所ですべてを持っている。`次は Google からの健康なキャッサバの葉のイメージです` 

![画像](https://cdn.shortpixel.ai/client/to_avif,q_lossless,ret_img,w_795,h_532/https://organic.ng/wp-content/uploads/2017/02/CASSAVA-LEAF.jpg)

* 上の画像から、健康的なキャッサバの葉の特徴の一つは、多くのカット、テクスチャの変更、黄色がかったグラデーション、等なしでかなり緑と直立する必要があることを言うことができます。

次に、データセットの中の健康なものを見て、上の画像と密接に混ざっているかどうかを見てみましょう。

In [ ]:
train[train['label_names']=='Healthy']['image_id'].count()

* 21kの画像のうち、2577だけが健康的(Healthy)なものである、ラベルの不均衡は明らかに目に見えている

In [ ]:
plot_images(class_id=4, 
    label='Healthy',
    images_number=6,verbose=1)

In [ ]:
plot_images(class_id=4, 
    label='Healthy',
    images_number=6,verbose=1)

上記の関数を3～4回実行して、毎回新しい画像が表示されるのを注意深く観察すると、次のようなことに気づくでしょう。
* すべての画像が葉をクローズアップしているわけではありませんし、人間の目には葉がほとんど見えない木全体が写っている画像もありますし、葉よりも茎が多く写っている画像もあります。
* さらに驚くべきことは、健康な葉の画像の中には感染しているように見えるものもあり、黄色や黄色がかったグラデーションのような色をしているものもあります。

### 外れ値の調査 :  
ポイント2を調査するために、私は次のようなアイデアを持っています。

* ここでのアイデアは、健全な画像をクラスタリングし、それぞれのクラスタを見て、外れ値クラスタと破損クラスタを見つけることができるかどうかを確認することです。
* クラスタリングのための特徴量を生成するために Resnet18 を使用します。

In [ ]:
def extract_features(image_id, model):
    file = BASE_DIR/'train_images'/image_id
    # load the image as a 224x224 array
    # load_imgはkerasの関数で出力はPILのインスタンス
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    # Numpy配列を前処理（モデルに画像のデータを入れる際に必要な処理か）
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    #引数で定めたモデルで予測する
    features = model.predict(imgx, use_multiprocessing=True)
    
    return features

In [ ]:
model = ResNet50()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

healthy = train[train['label']==4]
healthy['features'] = healthy['image_id'].progress_apply(lambda x:extract_features(x,model))

In [ ]:
model.summary()

In [ ]:
#画像数*2048
features.shape

In [ ]:
features = np.array(healthy['features'].values.tolist()).reshape(-1,2048)
image_ids = np.array(healthy['image_id'].values.tolist())

# Kmeansでクラスタリング(healtyの画像のみで)
kmeans = KMeans(n_clusters=5,n_jobs=-1, random_state=22)
kmeans.fit(features)

In [ ]:
#クラスタリングした際のラベル
kmeans.labels_

In [ ]:
groups = {}
for file, cluster in zip(image_ids,kmeans.labels_):
    if cluster not in groups.keys():
        #groupsというdictに、keyをcluster,valueを画像のファイル名を集めたリストとして格納
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 25")
        start = np.random.randint(0,len(files))
        files = files[start:start+25]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(5,5,index+1);
        img = load_img(BASE_DIR/'train_images'/file)
        img = np.array(img)
        plt.imshow(img)
        plt.title(file)
        plt.axis('off')

In [ ]:
view_cluster(3)

* クラスター3のほとんどの外れ値をクラスター化することができ、それらを簡単に可視化することができました。

* 葉が傷んでいたり、茶色い斑点があったり、健康ではないように見えるものもあります。

同じトピックに対処する多数の議論があります。
* https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198363 -- Wrong Labels
* https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/199606 --  Quality of Labels

訓練セットのノイズを心配する必要はありませんが、しかし、もしノイズがテストセットに含まれていて、ラベリングが同様に行われている場合はどうでしょうか？, それは問題かもしれません、我々は我々が確信するまで、訓練セットから何かを削除することはできません


このセクションを要約すると

` 健康なキャッサバの葉の特徴`:
* 主に緑色で、直立していて、茶色の斑点がほとんどない。
* 黄色でも緑でも葉全体に均一な質感を与える

In [ ]:
view_cluster(2)

### 病気を知る1：キャッサバ菌病（CBB）について

今、私たちは健康なキャッサバの葉がどのように見えるかを知っているので、 最初の病気について学ぶために移動しましょう `CBBの症状`:

* 黒い葉の斑点や病斑、角張った葉の斑点、若葉の萎凋による葉の早枯れや脱落、重度の攻撃などがあります。

* 最初は葉脈によって制限された葉に角張った水浸しの斑点が発生し、葉の下の方にはっきりと見られます。斑点は急速に拡大し、特に葉縁に沿って合流し、褐色で黄色の縁取りをする（図1）。

* 斑点の中心部にクリーム色の白色の液滴が発生し、その後、黄色に変化します。

![図1](https://www.pestnet.org/fact_sheets/assets/image/cassava_bacterial_blight_173/thumbs/cassavabb_sml.jpg)
![図2](https://www.pestnet.org/fact_sheets/assets/image/cassava_bacterial_blight_173/thumbs/cassavabb2_sml.jpg)


詳細は [here](https://www.pestnet.org/fact_sheets/cassava_bacterial_blight_173.htm)

In [ ]:
plot_images(class_id=0, 
    label='CBB',
    images_number=6,verbose=1)

* 症状の知識から、これらはCBB病に罹患していることは間違いありませんし、葉そのものではなく茎の画像を取得することは、病気の中には茎で判断できるものもありますので、茎の画像はノイズではないかもしれません。

* IMG - '1926670152.jpg'のような画像のいくつかでは、茶色の斑点は非常に小さく、葉は健康なもののように見え、健康な画像の多くはまた、そのような小さな茶色を持っており、識別するのは難しいかもしれません

* このカテゴリの病気の私の理解から、私はRandomCropping、コントラストの変化、任意の種類の色の変化は良いアイデアではないかもしれないと言うことができます。

### 病気について学ぶ2：キャッサバグリーンモット（CGM)

次の病気、 `Symptoms of CGM`に移ります

* 葉に白斑が発生し、最初の小さな斑点から葉全体に広がり、葉緑素が失われていきます。若い葉は凹み、かすかな黄色の斑点が目立つ（図1）。(図 1)

* この病気にかかると、葉に斑点状の症状が現れ、キャッサバモザイク病（CMD）の症状と混同されることがあります。重度のダメージを受けた葉は収縮し、乾燥して落ち、ローソク足のような特徴的な外観になります。(図2) (図 2)

![](https://www.pestnet.org/fact_sheets/assets/image/cassava_green_mottle_068/thumbs/cgmv2_sml.jpg)
![](https://www.pestnet.org/fact_sheets/assets/image/cassava_green_mottle_068/thumbs/cgmv_sml.jpg)

詳細は [here](https://www.pestnet.org/fact_sheets/cassava_green_mottle_068.htm)

In [ ]:
plot_images(class_id=2, 
    label='CGM',
    images_number=12,verbose=1)

#### 推論

* CGMの症状を読み、データセットの画像を見た後、CGMの葉、CBBの葉、健康な葉の違いを明確に伝えることができます。
* CGMの葉はビエン(viens)に沿って葉にかすかに黄色の斑点があり、CBBの葉は茶色の斑点があり、健康な葉は完全に緑か完全に黄色である。
* また、このクラスでもあまり外れた人はいません。


### 病気を知る3：キャッサバモザイク病（CMD）について

`CMDの症状`:

* CMDは、モザイク、斑入り、葉の変形やねじれ、葉や植物のサイズの全体的な減少を含む様々な葉状の症状を生成します。


* この病気によって影響を受けた葉は、通常の緑色のパッチを持ち、重症度に応じて黄色と白の異なる割合で混合されています。

In [ ]:
plot_images(class_id=3, 
    label='CMD',
    images_number=6,verbose=1)

### 推論

* 我々は、CGMとCMDは非常に近い症状を持っていることを見ることができ、また、かなり似たような画像を持っている、多くの場合、専門家は、これらのラベルを混乱させるかもしれません、我々はそれがモデルのためのものになりますどのように大きな課題を想像することができました。

* このカテゴリーでも外れ者はないか、あるいは非常に少ないように思われます。

### 病気を知る4：キャッサバ褐条病(CBSD)

今、私は最後にこれを選んだ理由は、我々はこのカテゴリのために2つの異なる種類の画像を持っているためです。

* 一つは、葉っぱ・植物の画像です。
* もう一つは、結節性の根の画像ですが、これはジャガイモやノイズと誤解されやすいのですが、データセットにノイズが含まれていることから、この病気の識別に偏りが出てしまいます。したがって、データセットに写っている茶色くて不恰好なものは、カサベの結節性根であり、この病気もまた、これらの画像から識別することができることを明確にしておきます。

今すぐ `CBSDの症状`を見てみましょう。

* CBSDの葉の症状は、比較的大きな黄色のパッチを形成するために拡大し、合体するかもしれない特徴的な黄色または壊死性の静脈のバンディングで構成されています。
* 塊根の症状は、塊茎内の黒褐色の壊死領域と根のサイズの減少で構成されています。

このカテゴリに存在する2つのタイプの画像と、それら2つの異なる画像に見られる症状を明確に理解することができましたので、データを見てみましょう。

In [ ]:
plot_images(class_id=1, 
    label='CBSD',
    images_number=12,verbose=1)

* 筒状の根の画像のクラスターをデータから取り出せるか試してみよう

In [ ]:
CBSD = train[train['label']==1]
CBSD['features'] = CBSD['image_id'].progress_apply(lambda x:extract_features(x,model))

In [ ]:
features_cbsd = np.array(CBSD['features'].values.tolist()).reshape(-1,2048)
image_ids_cbsd = np.array(CBSD['image_id'].values.tolist())

# Clustering
kmeans_cbsd = KMeans(n_clusters=5,n_jobs=-1, random_state=22)
kmeans_cbsd.fit(features_cbsd)

In [ ]:
groups_cbsd = {}
for file, cluster in zip(image_ids_cbsd,kmeans_cbsd.labels_):
    if cluster not in groups_cbsd.keys():
        groups_cbsd[cluster] = []
        groups_cbsd[cluster].append(file)
    else:
        groups_cbsd[cluster].append(file)

In [ ]:
def view_cluster(cluster):
    plt.figure(figsize = (25,25))
    # gets the list of filenames for a cluster
    files = groups_cbsd[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 25")
        start = np.random.randint(0,len(files))
        files = files[start:start+25]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(5,5,index+1);
        img = load_img(BASE_DIR/'train_images'/file)
        img = np.array(img)
        plt.imshow(img)
        plt.title(file)
        plt.axis('off')

In [ ]:
view_cluster(4)

* 我々は正常に80画像の1つのクラスタに管状根画像をクラスタ化することができました、それゆえに今、我々はすべてのIDSを取得することができ、この情報を使用する方法についての様々なアイデアを考えるかもしれません

In [ ]:
view_cluster(3)

# 私たちの調査結果の概要：ステップ1の終了

最初のEDAの結果をまとめてみましょう。

* 健康な画像は正しくラベル付けされていない可能性があり、間違ってラベル付けされた画像は5つのクラスターのうち3つのクラスターにあります。
* 完全に黄色の葉は、常に葉が潜在的な病気を持っていることを示していない可能性があります。
* 葉に茶色の斑点があるのは、キャッサバのバクテリア・ベト病(CBB)を示しています。 
* すべてのイメージに異なった背景およびスケールの変化があります 
* 画像は一日の異なる時間帯に撮影されているため、異なる照明と露出を持っています。
* キャッサバグリーンモットル(CGM)とキャッサバモザイク病(CMD)は、画像と同様に非常に似た症状を持っており、簡単に互いに誤認表示される可能性があります。また、キャッサバモザイク病の例が13kもあるので、モデルがCGMをCGMとラベル付けする際に最もミスが多い可能性が高いです。
* 一枚の画像/キャッサバの植物には複数の共起性疾患が含まれている可能性があります。モデルはラベル付けを混乱させる
* CBSDはデータセットの中に2種類の画像を持っていますが、1つは植物/葉の画像で、もう1つはジャガイモやランダムノイズと誤解されやすい根の画像です。


リーフドクターになるための最初のステップが完了した後、患者さんのことを理解し、様々な病気のことを理解することができました。このステップは、私たちはより良いソリューションを構築するためのユニークなソリューション/プランをデバイスに役立ちます。

<b>注：私はより多くの発見を続けるように、私はこのセクションでより多くのそのような知見を追加していきます。</b>

## データの重複  我々が見逃していたもの

議論の場を調べていたら、データセットの中に画像が重複している可能性について話している[この](https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198202) スレッドを見つけました。画像のデータセットの中に重複した画像があるという話はとても興味深いものです．

* 画像の完全なコピーについて話している
* 私たちは、特定の画像に似ている画像について話しています。例：画像1はトリミングされたか回転され、画像2として保存されています。

さて、画像データセットの中から重複画像（正確なコピー）や類似画像を見つけて識別する方法がいくつかあります。私は画像ハッシュ化の方法を使い、 [ここ](https://www.kaggle.com/appian/let-s-find-out-duplicate-images-with-imagehash) で見つけたノートに従っていきます。

In [ ]:
funcs = [
        imagehash.average_hash,
        imagehash.phash,
        imagehash.dhash,
        imagehash.whash,
    ]

image_ids = []
hashes = []

for path in tqdm(glob.glob(str(BASE_DIR/'train_images'/'*.jpg' ))):
    image = Image.open(path)
    image_id = os.path.basename(path)
    image_ids.append(image_id)
    hashes.append(np.array([f(image).hash for f in funcs]).reshape(256))

In [ ]:
hashes_all = np.array(hashes)

In [ ]:
hashes_all.shape

numpy配列をトーチテンソルに変換して類似度計算を高速化します。

In [ ]:
hashes_all = torch.Tensor(hashes_all.astype(int)).cuda()

すべての画像ペア間の類似度を計算します。値を256で割って正規化（0-1）します。

In [ ]:
%time sims = np.array([(hashes_all[i] == hashes_all).sum(dim=1).cpu().numpy()/256 for i in range(hashes_all.shape[0])])

In [ ]:
sims.shape

しきい値の設定

In [ ]:
indices1 = np.where(sims > 0.9)
indices2 = np.where(indices1[0] != indices1[1])
image_ids1 = [image_ids[i] for i in indices1[0][indices2]]
image_ids2 = [image_ids[i] for i in indices1[1][indices2]]
dups = {tuple(sorted([image_ids1,image_ids2])):True for image_ids1, image_ids2 in zip(image_ids1, image_ids2)}
print('found %d duplicates' % len(dups))

重複した画像のプロット

In [ ]:
'''
code taken from https://www.kaggle.com/nakajima/duplicate-train-images?scriptVersionId=47295222
'''

duplicate_image_ids = sorted(list(dups))

fig, axs = plt.subplots(2, 2, figsize=(15,15))

for row in range(2):
        for col in range(2):
            img_id = duplicate_image_ids[row][col]
            img = Image.open(str(BASE_DIR/'train_images'/img_id))
            label =str(train.loc[train['image_id'] == img_id].label.values[0])
            axs[row, col].imshow(img)
            axs[row, col].set_title("image_id : "+ img_id + "  label : " + label)
            axs[row, col].axis('off')

重複を見つける方法は他にもあり、データセットの中にソフトな重複がある場合には、このカーネルの後のバージョンで提供されます。

# ステップ2：方法論について学ぶ

こんにちは、医師はあなたの2年目へようこそ、あなたの最終課題を完了するために、あなたは今、あなたの処分で持っているツールを理解し、それらを使用する方法を理解する必要があります、以下は、ツールを学ぶために従うべきステップバイステップのガイドです。

* [Beginner Article](https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/)
* [Course By Andrew NG](https://www.coursera.org/learn/convolutional-neural-networks)
* [Applying CNNS using Keras and tensorflow](https://www.coursera.org/learn/convolutional-neural-networks-tensorflow)
* [Course from Fast.ai](https://course.fast.ai/videos/?lesson=1)

# ステップ3：最終プロジェクトの構築


Ohk now Docs , its time for you to build the final project . これは最終的なプロジェクトなので、誰もが自分自身で構築することを意味していますが、ここでは私が使用したものの要約を書き、さらにプロジェクトを改善するための方法を提案します。

最後に、私はまた、競争の全体のコースの中で試すためのもの/外を見るためのものを追加します。

`ベースラインモデルのまとめ`:

このモデルはキャッサバ2019大会の優勝解を元にしているので、できるだけ近い形で再現してみたいと思います。

* SE-ResNext50
* Dimension = (384,384)
* Epochs = 10
* Custom LR scheduler 
* Weights saved on best loss : Categorical CrossEntropy
* Basic Augs : HorizontalFlip,VerticalFlip,Rotate,RandomBrightness,ShiftScaleRotate,cutout,centercrop,zoom,randomscale
* No TTA（testデータにaugumentを行わない）

<font color ='red' >GPUのためのkaggleに制限されているので、私の5つ折りモデルはまだ実行されているので、今のところはSeResNext50の事前学習された重みだけを使用しています。このノートブックは、異なる設定/アイデアで数回更新されますので、チューニングを維持してください</color>

## 設定とユーティリティ機能

In [ ]:
DIM = (384,384)

NUM_WORKERS = 12
TEST_BATCH_SIZE = 16
SEED = 2020

DEVICE = "cuda"

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

## Augmentations

In [ ]:
def get_test_transforms():

    return albumentations.Compose(
        [albumentations.Normalize(MEAN, STD, max_pixel_value=255.0, always_apply=True),
        ToTensorV2(p=1.0)
        ]
    )

# Cassava Dataset

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self,image_ids,labels,dimension=None,augmentations=None):
        super().__init__()
        self.image_ids = image_ids
        self.labels = labels
        self.dim = dimension
        self.augmentations = augmentations
        
    def __len__(self):
        # len(上で定義したクラスのインスタンス)で返す値。
        return len(self.image_ids)
    
    def __getitem__(self,idx):
        # 上で定義したクラスのインスタンス[idx]で返す値
        img = cv2.imread(str(BASE_DIR/'test_images'/self.image_ids[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                         
        if self.dim:
            img = cv2.resize(img,self.dim)
        
        if self.augmentations:
            augmented = self.augmentations(image=img)
            image = augmented['image']
                         
        return {
            'image': image,
            'target': torch.tensor(self.labels[idx],dtype=torch.float)
        }

# Model : SE_Resnext50

In [ ]:
class CassavaModel(nn.Module):
    def __init__(self, model_name='seresnext50_32x4d',out_features=5,pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        
        #resnet50のモデルに最後出力の次元を揃えるために1層追加している。
        
        n_features = self.model.last_linear.in_features
        self.model.last_linear = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x

# Prediction Function Single Model

In [ ]:
def predict_single_model(data_loader,model,device):
    model.eval()
    tk0 = tqdm(enumerate(data_loader), total=len(data_loader))
    fin_out = []
    
    with torch.no_grad():
        
        for bi, d in tk0:
            images = d['image']
            targets = d['target']
            
            images = images.to(device)
            targets = targets.to(device)
            
            batch_size = images.shape[0]
            
            outputs = model(images)
            
            fin_out.append(F.softmax(outputs, dim=1).detach().cpu().numpy())
            
    return np.concatenate(fin_out)

# Engine

In [ ]:
sample_sub = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
def predict(weights):
    '''
    weights : List of paths in case of K fold model inference
    '''
    pred = np.zeros((len(sample_sub),5,5))
    
    # Defining DataSet
    test_dataset = CassavaDataset(
        image_ids=sample_sub['image_id'].values,
        labels=sample_sub['label'].values,
        augmentations=get_test_transforms(),
        dimension = DIM
    )
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=TEST_BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=False,
        pin_memory=True,
        drop_last=False,
    )
    
    # Defining Device
    device = torch.device("cpu")
    
    for i,weight in enumerate(weights):
        # Defining Model for specific fold
        model = CassavaModel(out_features=5,pretrained=True)
        
        # loading weights
        #model.load_state_dict(torch.load(weight))
        model.to(device)
        
        #predicting
        pred[:,:,i] = predict_single_model(test_loader,model,device)
    
    return pred

# Preparing Final Submission

In [ ]:
pred = predict([1])
print(pred)

In [ ]:
pred = pred.mean(axis=-1)
print('Prediction Before Argmax',pred)
pred = pred.argmax(axis=1)
print('Final Prediction',pred)

In [ ]:
sample_sub['label'] = pred
sample_sub.head()

In [ ]:
sample_sub.to_csv('submission.csv',index=False)

# 結論

競争が始まったばかりなので、試してみることがたくさんありますが、私はこのノートを更新してみます。

私のノートを読んでくれてありがとう , 私はあなたがそれから有用な何かを得たことを願っています。